In [ ]:
#default_exp model_hub

# Interacting with HuggingFace and Flair, Model Zoo
> An interactive API for model lookup within HuggingFace and Flair

In [ ]:
#hide
from nbdev.showdoc import show_doc
from fastcore.test import *

In [ ]:
#export
from fastcore.basics import Self, merge
from fastcore.utils import dict2obj, obj2dict
from fastai_minima.utils import apply
from huggingface_hub.hf_api import ModelInfo, HfApi

from typing import List, Dict

In [ ]:
#exporti
_hf_tasks = {
    'FILL_MASK':'fill-mask',
    'QUESTION_ANSWERING':'question-answering',
    'SUMMARIZATION':'summarization',
    'TABLE_QUESTION_ANSWERING':'table-question-answering',
    'TEXT_CLASSIFICATION':'text-classification',
    'TEXT_GENERATION':'text-generation',
    'TEXT2TEXT_GENERATION':'text2text-generation',
    'TOKEN_CLASSIFICATION':'token-classification',
    'TRANSLATION':'translation',
    'ZERO_SHOT_CLASSIFICATION':'zero-shot-classification',
    'CONVERSATIONAL':'conversational',
    'TEXT_TO_SPEECH':'text-to-speech',
    'AUTOMATIC_SPEECH_RECOGNITION':'automatic-speech-recognition',
    'AUDIO_SOURCE_SEPERATION':'audio-source-seperation',
    'VOICE_ACTIVITY_DETECTION':'voice-activity-detection'}

In [ ]:
#export
HF_TASKS = dict2obj(_hf_tasks)
HF_TASKS.__doc__ = "A list of all HuggingFace tasks for valid API lookup"

For convience the `HF_TASKS` and `FLAIR_TASKS` objects can enable tab-completion for searching through the API:

In [ ]:
show_doc(HF_TASKS)

<h4 id="- FILL_MASK: fill-mask
- QUESTION_ANSWERING: question-answering
- SUMMARIZATION: summarization
- TABLE_QUESTION_ANSWERING: table-question-answering
- TEXT_CLASSIFICATION: text-classification
- TEXT_GENERATION: text-generation
- TEXT2TEXT_GENERATION: text2text-generation
- TOKEN_CLASSIFICATION: token-classification
- TRANSLATION: translation
- ZERO_SHOT_CLASSIFICATION: zero-shot-classification
- CONVERSATIONAL: conversational
- TEXT_TO_SPEECH: text-to-speech
- AUTOMATIC_SPEECH_RECOGNITION: automatic-speech-recognition
- AUDIO_SOURCE_SEPERATION: audio-source-seperation
- VOICE_ACTIVITY_DETECTION: voice-activity-detection" class="doc_header"><code>- FILL_MASK: fill-mask
- QUESTION_ANSWERING: question-answering
- SUMMARIZATION: summarization
- TABLE_QUESTION_ANSWERING: table-question-answering
- TEXT_CLASSIFICATION: text-classification
- TEXT_GENERATION: text-generation
- TEXT2TEXT_GENERATION: text2text-generation
- TOKEN_CLASSIFICATION: token-classification
- TRANSLATION: translation
- ZERO_SHOT_CLASSIFICATION: zero-shot-classification
- CONVERSATIONAL: conversational
- TEXT_TO_SPEECH: text-to-speech
- AUTOMATIC_SPEECH_RECOGNITION: automatic-speech-recognition
- AUDIO_SOURCE_SEPERATION: audio-source-seperation
- VOICE_ACTIVITY_DETECTION: voice-activity-detection</code><a href="" class="source_link" style="float:right">[source]</a></h4>

A list of all HuggingFace tasks for valid API lookup

In [ ]:
#export
class HFModelResult:
    """
    A very basic class for storing a HuggingFace model returned through an API request
    """
    def __init__(self, model_info: ModelInfo):
        self.info = model_info
        
    def __repr__(self): return f'Model Name: {self.name}, Tasks: [' + ', '.join(self.tasks) + ']'
    
    @property
    def name(self): return self.info.modelId

    @property
    def tags(self): return self.info.tags
    
    @property
    def tasks(self): 
        if self.info.pipeline_tag:
            all_tasks = [self.info.pipeline_tag]
            for tag in self.tags:
                if (tag in HF_TASKS.keys()) and (tag not in all_tasks):
                    all_tasks += [tag]
        else: all_tasks = []
        return all_tasks
    
    def to_dict(self): return {'model_name':self.name, 'tags':self.tags, 'tasks':self.tasks, 'model_info':self.info}

In [ ]:
#export
class HFModelSearchHub:
    """
    A class for interacting with the HF model hub API, and searching for models by name or task
    
    Needs username and password for a `HuggingFace` account to search
    """
    
    def __init__(self, username='', password=''):
        self.api = HfApi()
        self.token = self.api.login(username, password)
        
    def _format_results(self, results:list, as_dict=False, user_uploaded=False) -> (List[HFModelResult], Dict[str, HFModelResult]):
        """
        Takes raw HuggingFace API results and makes them easier to read and work with
        """
        results = apply(HFModelResult, results)
        if not user_uploaded:
            results = [r for r in results if '/' not in r.name]
        if as_dict:
            dicts = apply(Self.to_dict(), results)
            results = {m['model_name'] : m for m in dicts}
        return results
        
    def search_model_by_task(self, task:str, as_dict=False, user_uploaded=False) -> (List[HFModelResult], Dict[str, HFModelResult]):
        """
        Searches HuggingFace Model API for all pretrained models relating to `task` and returns a list of HFModelResults
        
        Optionally can return all models as a `dict` rather than a list
        
        If `user_uploaded` is False, will only return models originating in HuggingFace (such as distilgpt2)
              
        Usage:
        ```python
          hub = HFModelHubSearch(username, password)
          hub.search_model_by_task('summarization')
          # OR #
          hub.search_model_by_task(HF_TASKS.SUMMARIZATION)
      ```
        """
        if task not in TASKS.values():
            raise ValueError(f'''`{task}` is not a valid task. 
            
            Please choose a valid one available from HuggingFace: (https://huggingface.co/transformers/task_summary.html) 
            Or with the `HF_TASKS` object''')
        models = self.api.list_models(task)
        return self._format_results(models, as_dict, user_uploaded)
    
    def search_model_by_name(self, name:str, as_dict=False, user_uploaded=False) -> (List[HFModelResult], Dict[str, HFModelResult]):
        """
        Searches HuggingFace Model API for all pretrained models containing `name` and returns a list of HFModelResults
        
        Optionally can return all models as `dict` rather than a list
        
        If `user_uploaded` is False, will only return models originating from HuggingFace (such as distilgpt2)
        
        Usage:
          ```python
          hub = HFModelHubSearch(username, password)
          hub.search_model_by_name('gpt2')
          ```
        """
        if user_uploaded:
            models = self.api.list_models()
            models = self._format_results(models, as_dict, user_uploaded)
            models = [m for m in models if name in m.name]
            
        else:
            models = self.api.list_models(name)
            models = self._format_results(models, as_dict, user_uploaded)
        return models

In [ ]:
#exporti
_flair_tasks = {
    'NAMED_ENTITY_RECOGNITION':'ner',
    'PHRASE_CHUNKING':'chunk',
    'VERB_DISAMBIGUATION':'frame',
    'PART_OF_SPEECH_TAGGING':'pos',
    'UNIVERSAL_PART_OF_SPEECH_TAGGING':'upos'
    
}

In [ ]:
#export
FLAIR_TASKS = dict2obj(_flair_tasks)
FLAIR_TASKS.__doc__ = "A list of all Flair model tasks for valid API lookup"

In [ ]:
#export
class FlairModelSearchHub:
    """
    A class for interacting with the HF model hub API, and searching for Flair models by name or task
    
    Needs username and password for a `HuggingFace` account to search
    """
    
    def __init__(self, username='', password=''):
        self.api = HfApi()
        self.token = self.api.login(username, password)
        
    def _format_results(self, results:list, as_dict=False, user_uploaded=False) -> (List[HFModelResult], Dict[str, HFModelResult]):
        """
        Takes raw HuggingFace API results and makes them easier to read and work with
        """
        results = apply(HFModelResult, results)
        if not user_uploaded:
            results = [r for r in results if 'flair/' in r.name]
        if as_dict:
            dicts = apply(Self.to_dict(), results)
            results = {m['model_name'] : m for m in dicts}
        return results
    
    def search_model_by_name(self, name:str, as_dict=False, user_uploaded=False) -> (List[HFModelResult], Dict[str, HFModelResult]):
        """
        Searches HuggingFace Model API for all flair models containing `name` and returns a list of `HFModelResults`
        
        Optionally can return all models as `dict` rather than a list
        
        If `user_uploaded` is False, will only return models originating from Flair (such as flair/chunk-english-fast)
        
        Usage:
          ```python
          hub = FlairModelHubSearch(username, password)
          hub.search_model_by_name('flair/chunk-english-fast')
          ```
        """
        models = self.api.list_models('flair')
        models = [m for m in models if name in m.modelId]
        return self._format_results(models, as_dict, user_uploaded)
    
    def search_model_by_task(self, task:str, as_dict=False, user_uploaded=False) -> (List[HFModelResult], Dict[str, HFModelResult]):
        """
        Searches HuggingFace Model API for all flair models for `task` and returns a list of `HFModelResults`
        
        Optionally can return all models as `dict` rather than a list
        
        If `user_uploaded` is False, will only return models originating from Flair (such as flair/chunk-english-fast)
        
        Usage:
        ```python
            hub = FlairModelHubSearch(username, password)
            hub.search_model_by_task('ner')
            # OR: #
            hub.search_model_by_task(FLAIR_TASKS.NAMED_ENTITY_RECOGNITION)
        """
        if (task not in FLAIR_TASKS.values()) and (task != ''):
            raise ValueError(f'''`{task}` is not a valid task. 
            
            Please choose a valid one available from Flair: (https://huggingface.co/flair) 
            Or with the `FLAIR_TASKS` object''')
        models = self.api.list_models('flair')
        models = [m for m in models if task in m.modelId]
        return self._format_results(models, as_dict, user_uploaded)

In [ ]:
# TODO: Write tests with a gh secret so I can test the API